# Reshaping a DataFrame with Pandas stack() and unstack()

- `stack()`: stack the prescribed level(s) from column to row.
- `unstack()`: unstack the prescribed level(s) from row to column. The inverse operation from stack.

<img src='./pandasData/stackunstack.png' style='width: 600px;'>

- Single level
- Multiple levels: simple case
- Multiple levels: missing values
- Multiple levels: specify a level to stack
- Multiple levels: drop missing values
- unstack: simple case
- unstack: more levels

In [1]:
# %load command1.py
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

%config InlineBackend.figure_format='svg'
plt.rcParams['figure.dpi']=120

pd.options.display.float_format='{:,.2f}'.format
pd.set_option('display.max_colwidth', None)


In [2]:
print(pd.__version__)
print(np.__version__)

1.4.2
1.21.5


**Single level**

In [3]:
df_single_level = pd.DataFrame(
    [['Mostly cloudy', 10], ['Sunny', 12]],
    index=['London', 'Oxford'],
    columns=['Weather', 'Wind']
)

df_single_level

df_single_level.stack()

,Weather,Wind
London,Mostly cloudy,10
Oxford,Sunny,12


London  Weather    Mostly cloudy
        Wind                  10
Oxford  Weather            Sunny
        Wind                  12
dtype: object

**Multiple levels:simple case**

In [4]:
multi_col_1 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Wind', 'm/s')]
)
df_multi_level_1 = pd.DataFrame(
    [[13, 5.5], [19, 8.5]],
    index=['London', 'Oxford'],
    columns=multi_col_1
)

df_multi_level_1
print()

df_multi_level_1.stack()

# Same as 
df_multi_level_1.stack(level=-1)
df_multi_level_1.stack(-1)

Wind     
        mph  m/s
London   13 5.50
Oxford   19 8.50

Wind
London m/s  5.50
       mph 13.00
Oxford m/s  8.50
       mph 19.00

Wind
London m/s  5.50
       mph 13.00
Oxford m/s  8.50
       mph 19.00

Wind
London m/s  5.50
       mph 13.00
Oxford m/s  8.50
       mph 19.00

**Multiple levels: missing values**

In [5]:
multi_col_2 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Temperature', '°C')]
)
df_multi_level_2 = pd.DataFrame(
    [[13, 8], [19, 6]],
    index=['London', 'Oxford'],
    columns=multi_col_2
)

df_multi_level_2
print()
df_multi_level_2.stack()

,Wind,Temperature
,mph,°C
London,13,8
Oxford,19,6


Temperature  Wind
London mph          NaN 13.00
       °C          8.00   NaN
Oxford mph          NaN 19.00
       °C          6.00   NaN

**Multiple levels: prescribing the level(s) to be stacked**

In [6]:
multi_col_2 = pd.MultiIndex.from_tuples(
    [('Wind', 'mph'), ('Temperature', '°C')]
)

# Level 0
multi_col_2.get_level_values(0)
# Index(['Wind', 'Temperature'], dtype='object')

# Level 1
multi_col_2.get_level_values(1)
# Index(['mph', '°C'], dtype='object')

Index(['Wind', 'Temperature'], dtype='object')

Index(['mph', '°C'], dtype='object')

In [7]:
df_multi_level_2.stack(0)
print()
df_multi_level_2.stack([0, 1])
print()
df_multi_level_2.stack([1, 0])

mph   °C
London Temperature   NaN 8.00
       Wind        13.00  NaN
Oxford Temperature   NaN 6.00
       Wind        19.00  NaN

London  Temperature  °C     8.00
        Wind         mph   13.00
Oxford  Temperature  °C     6.00
        Wind         mph   19.00
dtype: float64

London  mph  Wind          13.00
        °C   Temperature    8.00
Oxford  mph  Wind          19.00
        °C   Temperature    6.00
dtype: float64

**Multiple levels: dropping missing values**

In [8]:
df_multi_level_3 = pd.DataFrame(
    [[None, 10], [11, 7.0]],
    index=['London', 'Oxford'],
    columns=multi_col_2
)

df_multi_level_3
df_multi_level_3.stack()
df_multi_level_3.stack(dropna=False)

,Wind,Temperature
,mph,°C
London,NaN,10.00
Oxford,11.00,7.00


Temperature  Wind
London °C         10.00   NaN
Oxford mph          NaN 11.00
       °C          7.00   NaN

Temperature  Wind
London mph          NaN   NaN
       °C         10.00   NaN
Oxford mph          NaN 11.00
       °C          7.00   NaN

**unstack: simple case**

In [9]:
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Temperature'), 
  ('Oxford', 'Wind'),
  ('London', 'Temperature'), 
  ('London', 'Wind')
])
s = pd.Series([1,2,3,4], index=index)
s

Oxford  Temperature    1
        Wind           2
London  Temperature    3
        Wind           4
dtype: int64

In [10]:
s.unstack()
# It's equivalent to
s.unstack(level=-1)

# Unstack a specific level
s.unstack(level=0)

,Temperature,Wind
London,3,4
Oxford,1,2


,Temperature,Wind
London,3,4
Oxford,1,2


,London,Oxford
Temperature,3,1
Wind,4,2


**unstack: more levels**

In [11]:
index = pd.MultiIndex.from_tuples([
  ('Oxford', 'Weather', '01-01-2022'), 
  ('Oxford', 'Temperature', '01-01-2022'), 
  ('Oxford', 'Weather', '02-01-2022'),
  ('Oxford', 'Temperature', '02-01-2022'),
  ('London', 'Weather', '01-01-2022'), 
  ('London', 'Temperature', '01-01-2022'),
  ('London', 'Weather', '02-01-2022'),
  ('London', 'Temperature', '02-01-2022'),
])
s = pd.Series(
  ['Sunny', 10, 'Shower', 7, 'Shower', 5, 'Sunny', 8], 
  index=index
)

s
print()

Oxford  Weather      01-01-2022     Sunny
        Temperature  01-01-2022        10
        Weather      02-01-2022    Shower
        Temperature  02-01-2022         7
London  Weather      01-01-2022    Shower
        Temperature  01-01-2022         5
        Weather      02-01-2022     Sunny
        Temperature  02-01-2022         8
dtype: object

In [12]:
# Method chaining
s.unstack().unstack()
print()

# The equivalent
s.unstack([2,1])

01-01-2022          02-01-2022        
       Temperature Weather Temperature Weather
London           5  Shower           8   Sunny
Oxford          10   Sunny           7  Shower

01-01-2022             02-01-2022            
          Weather Temperature    Weather Temperature
London     Shower           5      Sunny           8
Oxford      Sunny          10     Shower           7

In [13]:
s.unstack().unstack().unstack()
print()

01-01-2022  Temperature  London         5
                         Oxford        10
            Weather      London    Shower
                         Oxford     Sunny
02-01-2022  Temperature  London         8
                         Oxford         7
            Weather      London     Sunny
                         Oxford    Shower
dtype: object

In [14]:
# The equivalent
s.unstack(level=[0,1])

Oxford              London            
           Weather Temperature Weather Temperature
01-01-2022   Sunny          10  Shower           5
02-01-2022  Shower           7   Sunny           8